In [1]:
import glob
import pandas as pd
import numpy as np
import string

In [2]:
file = "state_lat_lng.csv"
read_file = pd.read_csv(file, encoding='latin1')
state_df = pd.DataFrame(read_file)

file = "combined_data.csv"
read_file = pd.read_csv(file, encoding='latin1')
df = pd.DataFrame(read_file)

In [3]:
df.head()

,Unnamed: 0,Years,Cancer Type,State,County,FIPS,Met Health Obj,Age-Adjusted Incident Rate,Lower 95% Confidence Interval,Lower 95% Confidence Interval.1,Upper 95% Confidence Interval,Upper 95% Confidence Interval.1,Average Annual Count,Recent Trend,Recent 5-Year Trend in Incidence Rates,Lower 95% Confidence Interval.2,Lower 95% Confidence Interval.3,Upper 95% Confidence Interval.2,Upper 95% Confidence Interval.3
0,8,2011-2015,Breast,Alabama,Alabama,1000,***,120.9,119.1,0.2,122.7,1.7,3645,rising,1,119.1,0.2,122.7,1.7
1,9,2011-2015,Breast,Alabama,US,0,***,124.7,124.4,-0.2,124.9,0.6,234445,stable,0.2,124.4,-0.2,124.9,0.6
2,10,2011-2015,Breast,Alabama,Pickens County,1107,***,162.1,130.6,-29.7,199.2,56.4,20,stable,4.8,130.6,-29.7,199.2,56.4
3,11,2011-2015,Breast,Alabama,Conecuh County,1035,***,162.1,123.4,-18,209.5,7.9,14,stable,-6,123.4,-18,209.5,7.9
4,12,2011-2015,Breast,Alabama,Bibb County,1007,***,157.9,128.2,-39.6,192.7,53.3,21,stable,-3.8,128.2,-39.6,192.7,53.3


In [4]:
same_df = df[df['State'] == df['County']]
breast_df        = same_df.set_index('Cancer Type').filter(like='Breast', axis=0)
colon_df         = same_df.set_index('Cancer Type').filter(like='Colon & Rectum', axis=0)
lung_df          = same_df.set_index('Cancer Type').filter(like='Lung & Bronchus', axis=0)
prostate_df      = same_df.set_index('Cancer Type').filter(like='Prostate', axis=0)

In [5]:
colon_df.head()

,Unnamed: 0,Years,State,County,FIPS,Met Health Obj,Age-Adjusted Incident Rate,Lower 95% Confidence Interval,Lower 95% Confidence Interval.1,Upper 95% Confidence Interval,Upper 95% Confidence Interval.1,Average Annual Count,Recent Trend,Recent 5-Year Trend in Incidence Rates,Lower 95% Confidence Interval.2,Lower 95% Confidence Interval.3,Upper 95% Confidence Interval.2,Upper 95% Confidence Interval.3
Cancer Type,,,,,,,,,,,,,,,,,,
Colon & Rectum,8,2011-2015,Alabama,Alabama,1000,No,43.5,42.7,-2.3,44.3,3.1,2458,stable,0.3,42.7,-2.3,44.3,3.1
Colon & Rectum,8,2011-2015,Alaska,Alaska,2900,No,42,39.6,-1.5,44.5,4.2,269,stable,1.3,39.6,-1.5,44.5,4.2
Colon & Rectum,8,2011-2015,Arizona,Arizona,4000,Yes,33.6,33,-4.3,34.2,-0.1,2564,falling,-2.3,33,-4.3,34.2,-0.1
Colon & Rectum,8,2011-2015,Arkansas,Arkansas,5000,No,43.6,42.6,1,44.6,2.8,1515,rising,1.9,42.6,1,44.6,2.8
Colon & Rectum,8,2011-2015,Colorado,Colorado,8000,Yes,33.8,33.1,-2.9,34.5,-0.2,1812,falling,-1.6,33.1,-2.9,34.5,-0.2


In [6]:
state_df.head()

,State,Latitude,Longitude
0,Alabama,32.806671,-86.791130
1,Alaska,61.370716,-152.404419
2,Arizona,33.729759,-111.431221
3,Arkansas,34.969704,-92.373123
4,California,36.116203,-119.681564


In [7]:
breast_joined_df   = state_df.join(breast_df.set_index('State'), on='State')
colon_joined_df    = state_df.join(colon_df.set_index('State'), on='State')
lung_joined_df     = state_df.join(lung_df.set_index('State'), on='State')
prostate_joined_df = state_df.join(prostate_df.set_index('State'), on='State')

In [11]:
lung_joined_df

,State,Latitude,Longitude,Unnamed: 0,Years,County,FIPS,Met Health Obj,Age-Adjusted Incident Rate,Lower 95% Confidence Interval,Lower 95% Confidence Interval.1,Upper 95% Confidence Interval,Upper 95% Confidence Interval.1,Average Annual Count,Recent Trend,Recent 5-Year Trend in Incidence Rates,Lower 95% Confidence Interval.2,Lower 95% Confidence Interval.3,Upper 95% Confidence Interval.2,Upper 95% Confidence Interval.3
0,Alabama,32.806671,-86.791130,8.0,2011-2015,Alabama,1000.0,***,67.7,66.8,-3.4,68.7,-2.7,3967,falling,-3,66.8,-3.4,68.7,-2.7
1,Alaska,61.370716,-152.404419,8.0,2011-2015,Alaska,2900.0,***,57.2,54.3,-8,60.2,-0.8,342,falling,-4.5,54.3,-8,60.2,-0.8
2,Arizona,33.729759,-111.431221,8.0,2011-2015,Arizona,4000.0,***,49.3,48.6,-4.1,50,-2.2,3902,falling,-3.1,48.6,-4.1,50,-2.2
3,Arkansas,34.969704,-92.373123,8.0,2011-2015,Arkansas,5000.0,***,78,76.7,-1.8,79.3,2.2,2818,stable,0.2,76.7,-1.8,79.3,2.2
4,California,36.116203,-119.681564,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Colorado,39.059811,-105.311104,8.0,2011-2015,Colorado,8000.0,***,43.3,42.5,-4.4,44.1,-1.5,2295,falling,-3,42.5,-4.4,44.1,-1.5
6,Connecticut,41.597782,-72.755371,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Delaware,39.318523,-75.507141,8.0,2011-2015,Delaware,10000.0,***,71.2,69,-5.5,73.4,3,822,stable,-1.4,69,-5.5,73.4,3
8,District of Columbia,38.897438,-77.026817,8.0,2011-2015,District of Columbia,11001.0,***,53.4,50.8,-15.2,56.1,0.8,331,stable,-7.5,50.8,-15.2,56.1,0.8
9,Florida,27.766279,-81.686783,8.0,2011-2015,Florida,12000.0,***,59.7,59.3,-5.4,60.1,-1.7,16548,falling,-3.6,59.3,-5.4,60.1,-1.7


In [10]:
colon_joined_df

,State,Latitude,Longitude,Unnamed: 0,Years,County,FIPS,Met Health Obj,Age-Adjusted Incident Rate,Lower 95% Confidence Interval,Lower 95% Confidence Interval.1,Upper 95% Confidence Interval,Upper 95% Confidence Interval.1,Average Annual Count,Recent Trend,Recent 5-Year Trend in Incidence Rates,Lower 95% Confidence Interval.2,Lower 95% Confidence Interval.3,Upper 95% Confidence Interval.2,Upper 95% Confidence Interval.3
0,Alabama,32.806671,-86.791130,8.0,2011-2015,Alabama,1000.0,No,43.5,42.7,-2.3,44.3,3.1,2458,stable,0.3,42.7,-2.3,44.3,3.1
1,Alaska,61.370716,-152.404419,8.0,2011-2015,Alaska,2900.0,No,42,39.6,-1.5,44.5,4.2,269,stable,1.3,39.6,-1.5,44.5,4.2
2,Arizona,33.729759,-111.431221,8.0,2011-2015,Arizona,4000.0,Yes,33.6,33,-4.3,34.2,-0.1,2564,falling,-2.3,33,-4.3,34.2,-0.1
3,Arkansas,34.969704,-92.373123,8.0,2011-2015,Arkansas,5000.0,No,43.6,42.6,1,44.6,2.8,1515,rising,1.9,42.6,1,44.6,2.8
4,California,36.116203,-119.681564,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Colorado,39.059811,-105.311104,8.0,2011-2015,Colorado,8000.0,Yes,33.8,33.1,-2.9,34.5,-0.2,1812,falling,-1.6,33.1,-2.9,34.5,-0.2
6,Connecticut,41.597782,-72.755371,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Delaware,39.318523,-75.507141,8.0,2011-2015,Delaware,10000.0,Yes,37.2,35.6,-4.2,38.8,6.7,415,stable,1.1,35.6,-4.2,38.8,6.7
8,District of Columbia,38.897438,-77.026817,8.0,2011-2015,District of Columbia,11001.0,No,40.9,38.6,-17.9,43.3,9.7,253,stable,-5.1,38.6,-17.9,43.3,9.7
9,Florida,27.766279,-81.686783,8.0,2011-2015,Florida,12000.0,Yes,36.9,36.5,-4.2,37.2,-1.2,9790,falling,-2.7,36.5,-4.2,37.2,-1.2
